In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)

def create_user_movie_df():
    import pandas as pd
    movie = pd.read_csv(r"C:\Users\ACER\Desktop\Miuul\Recommendation Systems\recommender_systems\datasets\movie_lens_dataset\movie.csv")
    rating = pd.read_csv(r"C:\Users\ACER\Desktop\Miuul\Recommendation Systems\recommender_systems\datasets\movie_lens_dataset\rating.csv")
    df = movie.merge(rating, how="left", on="movieId")
    comment_counts= pd.DataFrame(df["title"].value_counts())
    rare_movies = comment_counts[comment_counts["title"] <= 1000].index
    common_movies = df[~df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
    return user_movie_df

user_movie_df = create_user_movie_df()

In [2]:
import pandas as pd
random_user = int(pd.Series(user_movie_df.index).sample(1,random_state=79).values)
random_user_df = user_movie_df[user_movie_df.index == random_user]
movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()
movies_watched,random_user

(['Addams Family Values (1993)',
  'Aladdin (1992)',
  'Babe (1995)',
  'Batman (1989)',
  'Batman Forever (1995)',
  'Beauty and the Beast (1991)',
  'Clerks (1994)',
  'Clueless (1995)',
  'Coneheads (1993)',
  'Dances with Wolves (1990)',
  'Ed Wood (1994)',
  'Fargo (1996)',
  'Get Shorty (1995)',
  'Hoop Dreams (1994)',
  'I.Q. (1994)',
  'Indian in the Cupboard, The (1995)',
  'Like Water for Chocolate (Como agua para chocolate) (1992)',
  'Miracle on 34th Street (1994)',
  'Natural Born Killers (1994)',
  'Pulp Fiction (1994)',
  'Quiz Show (1994)',
  'Taxi Driver (1976)',
  'Toy Story (1995)',
  'Waterworld (1995)',
  "What's Eating Gilbert Grape (1993)"],
 57110)

In [3]:
len(movies_watched)

25

In [4]:
movies_watched_df = user_movie_df[movies_watched]
user_movie_count = movies_watched_df.T.notnull().sum()
user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["userId", "movie_count"]
user_movie_count

,userId,movie_count
0,1.0,3
1,2.0,0
2,3.0,3
3,4.0,0
4,5.0,6
...,...,...
138488,138489.0,0
138489,138490.0,9
138490,138491.0,1
138491,138492.0,2


In [5]:
users_same_movies = user_movie_count[user_movie_count["movie_count"] > (25*0.6)]["userId"]

In [6]:
final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
                      random_user_df[movies_watched]])


In [7]:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()
corr_df

,user_id_1,user_id_2,corr
0,64694.0,66317.0,-1.000000
1,34524.0,77588.0,-1.000000
2,20040.0,61012.0,-1.000000
3,86736.0,55804.0,-0.989371
4,54160.0,13059.0,-0.972111
...,...,...,...
8046252,129557.0,126739.0,1.000000
8046253,57110.0,57110.0,1.000000
8046254,35876.0,94179.0,1.000000
8046255,100429.0,117541.0,1.000000


In [8]:
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][
    ["user_id_2", "corr"]].reset_index(drop=True)
top_users = top_users.sort_values(by='corr', ascending=False)
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)
top_users

,userId,corr
5,57110.0,1.000000
4,21840.0,0.824364
3,74603.0,0.756778
2,83828.0,0.706535
1,9932.0,0.673581
0,126732.0,0.654101


In [9]:
rating = pd.read_csv(r"C:\Users\ACER\Desktop\Miuul\Recommendation Systems\recommender_systems\datasets\movie_lens_dataset\rating.csv")
top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')

top_users_ratings = top_users_ratings[top_users_ratings["userId"] != random_user]

In [10]:
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})

,weighted_rating
movieId,
1,1.959064
2,2.296348
3,2.826139
4,2.328314
5,2.119604
...,...
100032,2.885275
101904,2.885275
102194,3.297457


In [11]:
recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})
recommendation_df = recommendation_df.reset_index()

In [12]:
movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.5].sort_values("weighted_rating", ascending=False)
movie = pd.read_csv(r"C:\Users\ACER\Desktop\Miuul\Recommendation Systems\recommender_systems\datasets\movie_lens_dataset\movie.csv")


In [13]:
movies_to_be_recommend.merge(movie[["movieId", "title"]]).head()

,movieId,weighted_rating,title
0,3077,4.121821,42 Up (1998)
1,3072,4.121821,Moonstruck (1987)
2,3095,4.121821,"Grapes of Wrath, The (1940)"
3,3097,4.121821,"Shop Around the Corner, The (1940)"
4,3129,4.121821,Sweet and Lowdown (1999)


In [14]:
movie = pd.read_csv(r"C:\Users\ACER\Desktop\Miuul\Recommendation Systems\recommender_systems\datasets\movie_lens_dataset\movie.csv")
rating = pd.read_csv(r"C:\Users\ACER\Desktop\Miuul\Recommendation Systems\recommender_systems\datasets\movie_lens_dataset\rating.csv")
user= 57110

In [18]:
movie_id = rating[(rating["userId"] == user) & (rating["rating"] == 5.0)].sort_values(by="timestamp", ascending=False)["movieId"][0:1].values[0]
movie_id

223

In [19]:
movie_df = user_movie_df[movie[movie["movieId"] == movie_id]["title"].values[0]]
movie_df

userId
1.0         4.0
2.0         NaN
3.0         5.0
4.0         NaN
5.0         NaN
           ... 
138489.0    NaN
138490.0    NaN
138491.0    NaN
138492.0    NaN
138493.0    NaN
Name: Clerks (1994), Length: 138493, dtype: float64

In [21]:
user_movie_df.corrwith(movie_df).sort_values(ascending=False).iloc[1:11]



title
Clerks II (2006)                         0.490608
Mallrats (1995)                          0.467973
Chasing Amy (1997)                       0.446016
Jay and Silent Bob Strike Back (2001)    0.404679
Dogma (1999)                             0.380266
Woodstock (1970)                         0.356060
Underground (1995)                       0.328435
Zack and Miri Make a Porno (2008)        0.326978
Warriors, The (1979)                     0.322433
SLC Punk! (1998)                         0.320888
dtype: float64